<a href="https://colab.research.google.com/github/ANKITKISLAYA/Traffic-Sign-Classfier/blob/main/bottleneck_features_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from keras.applications import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.layers import Input, AveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.datasets import cifar10
import pickle
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import keras.backend as K

In [7]:

# Make bottleneck features for either cifar10 or traffic dataset.
dataset = 'cifar10'

# he model to bottleneck, ex. 'vgg', 'inception', or 'resnet
network = 'vgg'

# The batch size for the generator
batch_size = 128


h, w, ch = 224, 224, 3
if network == 'inception':
    h, w, ch = 299, 299, 3
    from keras.applications.inception_v3 import preprocess_input

img_placeholder = tf.placeholder("uint8", (None, 32, 32, 3))
resize_op = tf.image.resize_images(img_placeholder, (h, w), method=0)

In [8]:
def gen(session, data, labels, batch_size):
    def _f():
        start = 0
        end = start + batch_size
        n = data.shape[0]

        while True:
            X_batch = session.run(resize_op, {img_placeholder: data[start:end]})
            X_batch = tf.keras.applications.vgg16.preprocess_input(X_batch)
            y_batch = labels[start:end]
            start += batch_size
            end += batch_size
            if start >= n:
                start = 0
                end = batch_size

            print(start, end)
            yield (X_batch, y_batch)

    return _f

In [9]:



def create_model():
    input_tensor = Input(shape=(h, w, ch))
    if network == 'vgg':
        model = VGG16(input_tensor=input_tensor, include_top=False)
        x = model.output
        x = AveragePooling2D((7, 7))(x)
        model = Model(model.input, x)
    elif network == 'inception':
        model = InceptionV3(input_tensor=input_tensor, include_top=False)
        x = model.output
        x = AveragePooling2D((8, 8), strides=(8, 8))(x)
        model = Model(model.input, x)
    else:
        model = ResNet50(input_tensor=input_tensor, include_top=False)
    return model




In [ ]:

if dataset == 'cifar10':
    (X_train, y_train), (_, _) = cifar10.load_data()
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
else:
    with open('data/train.p', mode='rb') as f:
        train = pickle.load(f)
    X_train, X_val, y_train, y_val = train_test_split(train['features'], train['labels'], test_size=0.33, random_state=0)

train_output_file = "{}_{}_{}.p".format(network, dataset, 'bottleneck_features_train')
validation_output_file = "{}_{}_{}.p".format(network, dataset, 'bottleneck_features_validation')

print("Resizing to", (w, h, ch))
print("Saving to ...")
print(train_output_file)
print(validation_output_file)

with tf.Session() as sess:
    K.set_session(sess)
    K.set_learning_phase(1)

    model = create_model()

    # The model.predict_generator function returns an array of predicted probabilities, where each row corresponds to
    # a sample in the test set, and each column represents the probability of the sample belonging to a specific class.
    print('Bottleneck training')
    train_gen = gen(sess, X_train, y_train, batch_size)
    bottleneck_features_train = model.predict_generator(train_gen(), X_train.shape[0])
    data = {'features': bottleneck_features_train, 'labels': y_train}
    pickle.dump(data, open(train_output_file, 'wb'))

    print('Bottleneck validation')
    val_gen = gen(sess, X_val, y_val, batch_size)
    bottleneck_features_validation = model.predict_generator(val_gen(), X_val.shape[0])
    data = {'features': bottleneck_features_validation, 'labels': y_val}
    pickle.dump(data, open(validation_output_file, 'wb'))

Resizing to (224, 224, 3)
Saving to ...
vgg_cifar10_bottleneck_features_train.p
vgg_cifar10_bottleneck_features_validation.p
Bottleneck training
128 256


<ipython-input-10-358e240e2257>:27: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_train = model.predict_generator(train_gen(), X_train.shape[0])


256 384
384 512
512 640
640 768
768 896
896 1024
1024 1152
1152 1280
1280 1408
1408 1536
1536 1664
1664 1792
1792 1920
1920 2048
2048 2176
2176 2304
2304 2432
2432 2560
2560 2688
2688 2816
2816 2944
2944 3072
3072 3200
3200 3328
3328 3456
3456 3584
3584 3712
3712 3840
3840 3968
3968 4096
4096 4224
4224 4352
4352 4480
4480 4608
4608 4736
4736 4864
4864 4992
4992 5120
5120 5248
5248 5376
5376 5504
5504 5632
5632 5760
5760 5888
5888 6016
6016 6144
6144 6272
6272 6400
6400 6528
6528 6656
6656 6784
6784 6912
6912 7040
7040 7168
7168 7296
7296 7424
7424 7552
7552 7680
7680 7808
7808 7936
7936 8064
8064 8192
8192 8320
8320 8448
8448 8576
8576 8704
8704 8832
8832 8960
8960 9088
9088 9216
9216 9344
9344 9472
9472 9600
9600 9728
9728 9856
9856 9984
9984 10112
10112 10240
10240 10368
10368 10496
10496 10624
10624 10752
10752 10880
10880 11008
11008 11136
11136 11264
11264 11392
11392 11520
11520 11648
11648 11776
11776 11904
11904 12032
12032 12160
12160 12288
12288 12416
12416 12544
12544 12672


In [1]:
import pickle
import collections, numpy as np

training_file = "vgg_cifar10_bottleneck_features_train.p"
validation_file = "vgg_cifar10_bottleneck_features_validation.p"

with open(training_file, 'rb') as f:
    train_data = pickle.load(f)
with open(validation_file, 'rb') as f:
    validation_data = pickle.load(f)

FileNotFoundError: ignored